In [3]:
import os
import random
from typing import List
import twscrape

In [9]:
def set_up_directories(keyword_eng: str, year: str):
    os.mkdir(keyword_eng + "_" + year)

    # single digit month
    for i in range(1,10):
        path = keyword_eng + "_" + year + "/" + "0" + str(i)
        os.mkdir(path) 

    # double digit month
    for j in range(10,13):
        path = keyword_eng + "_" + year + "/" + str(j)
        os.mkdir(path) 

set_up_directories("test", "2001")

In [12]:
# remove directories if needed
# !rm -rf test_2001/

In [18]:
# collect tweets for 10 randomly sampled days of month at randomly sampled time of day
def scrape_sampled_tweets(year: str, months: List[str], days_in_month: int, keyword_jp="", keyword_eng=""):
    range_days = list(range(1,days_in_month+1))
    range_times = list(range(0,24))
    dir_path = year if keyword_eng == "" else keyword_eng + "_" + year

    # sample 10 random days and times of day for each month
    for m in months: 
        month_path = dir_path + "/" + year + "-" + m
        days = sorted(random.sample(range_days, k=10))   # random days of month (no replacement)
        times = random.choices(range_times, k=10)   # random hours of day (replacement)
        
        # scrape tweets for the 10 picked days and times
        for t in range(10):     
            day = "0" + str(days[t]) if days[t] < 10 else str(days[t])
            time = "0" + str(times[t]) if times[t] < 10 else str(times[t])
            day_path = month_path + "/" + year + "-" + m + "-" + day + ".txt"
            command = 'twscrape search "' + keyword_jp + ' since:' + year + '-' + m + '-' + day + '_' + time + ':00:00_UTC lang:ja"　> ' + day_path + ' --limit=4500'
            os.system(command)

In [17]:
def concatenate_txt_files(year: str, keyword_eng=""):
  # concatenate .txt files into one file per month
  for root, dirs, files in os.walk("./" + year):
      for name in dirs:
        month_path = os.path.join(root, name)
        os.system("cat " + month_path + "/*.txt > " + name + ".txt")
  
  # concatenate month .txt files into one file for the year
  os.system("cat " + year + "-*.txt > " + keyword_eng + "_" + year + "-all.txt")

In [16]:
# scrape random sample of all tweets from given year
def scrape_tweets_general_year(year: str):
    # 28 day months
    scrape_sampled_tweets(year,["02"], 28)    # omit leap year 29th days for simplicity...?

    # 30 day months
    months_30 = ["04","06","09","11"]
    scrape_sampled_tweets(year,months_30, 30)

    # 31 day months
    months_31 = ["01","03","05","07","08","10","12"]
    scrape_sampled_tweets(year,months_31, 31)

In [ ]:
scrape_tweets_general_year("2022")
concatenate_txt_files("2022")

In [1]:
# scrape all tweets containing keyword in given year
def scrape_tweets_keyword(year: str, keyword_jp: str, keyword_eng: str):
    command = 'twscrape search "' + keyword_jp + ' since:' + year + '-01-01_00:00:00_UTC lang:ja"　> ' + keyword_eng + '_' + year + '.txt'
    os.system(command)

In [22]:
scrape_tweets_keyword("2022", "在日コリアン", "zainichi_test")
concatenate_txt_files("2022", "zainichi_test")